# Import

In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModelWithLMHead

import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

import os
import gc

import re

import pyphen

In [2]:
dic = pyphen.Pyphen(lang='pl')

In [3]:
tokenizer = AutoTokenizer.from_pretrained('flax-community/papuGaPT2',
                                          bos_token='<|startoftext|>', 
                                          eos_token='<|endoftext|>', 
                                          pad_token='<|pad|>')

model = AutoModelWithLMHead.from_pretrained('models/papuga-poems-01')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/marek/anaconda3/envs/nlp/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:907: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


## Verify model

In [4]:
prompt = "Litwo Ojczyzno moja ty jesteś jak zdrowie"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

sample_outputs = model.generate(
    generated,
    do_sample=True, 
    max_length=100, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=1
)


for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Litwo Ojczyzno moja ty jesteś jak zdrowie. Tak mi przytnę uszy Twe,
Żeś mię sławił, g’woli Twojej wiernego sługi
Jako żyw, jakoć mię w cale nie zawodzi,
Jako żyw litości w sercu czuje.
Wszytka rozkosz mą na tym świecie,
Alem ja Ci, litościwy niechaj będzie dana
I w Twoim kościele, i w Twoim domu
Nie będę potym animował myśli naszych.




In [5]:
model_g = model.to('cpu')

In [6]:
model.eval()
generated_num = 0
generated_list = []

filter_value = -float("Inf")

prompt = "Litwo Ojczyzno moja ty jesteś jak zdrowie\n"

entry_count=10
entry_length=200
top_p=0.75 #sort word probabilities descending order, sum upp to p dropping other words, keeping few
temperature=1. # higher more originlal values

In [23]:
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

outputs = model_g(generated, labels=generated)
loss, logits = outputs[:2]
logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

sorted_logits, sorted_indices = torch.sort(logits, descending=True)
cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

sorted_indices_to_remove = cumulative_probs > top_p
sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
sorted_indices_to_remove[..., 0] = 0

In [24]:
# logits_gr0_idx = (logits > 0).nonzero(as_tuple=True)[1]
# logits_gr0_idx

In [25]:
# logits_gr0 = logits[(logits > 0)]
# logits_gr0

In [26]:
# f_softmax = F.softmax(logits_gr0, dim=-1)
# f_softmax

In [29]:
f_softmax = F.softmax(logits, dim=-1)
indeces_sort = torch.argsort(f_softmax)[0,:50]
indeces_sort.shape

torch.Size([50])

In [56]:
indeces_sort = torch.argsort(f_softmax)
indeces_sort

tensor([ 9, 16, 20, 19,  1, 18, 11,  8,  5, 23, 14,  3, 22, 17, 27, 12, 15, 25,
         7,  6, 21, 13,  4, 26,  0, 24, 10,  2])

In [ ]:
indices_to_remove = sorted_indices[sorted_indices_to_remove]
logits[:, indices_to_remove] = filter_value

In [88]:
def get_next_token(generated, n_logits=50):
    outputs = model_g(generated, labels=generated)
    loss, logits = outputs[:2]
    logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    
    if True:
        f_softmax = F.softmax(logits, dim=-1)
        indeces_sort = torch.argsort(f_softmax)[0, :n_logits]
    
    logits[:, indices_to_remove] = filter_value
    next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
        
#     logits_gr0_idx = (logits > 0).nonzero(as_tuple=True)[1]
#     logits_gr0 = logits[(logits > 0)]
#     f_softmax = F.softmax(logits, dim=-1)
#     indeces_sort = torch.argsort(f_softmax)[:n_logits]

    return next_token, indeces_sort #, logits_gr0_idx

In [89]:
type_r = 'aabb'
if type_r == 'aabb':
    wersy_n = 1
else:
    wersy_n = 3

In [94]:
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

rhymes = []
if len(re.findall('\n', prompt)) > 0:
    word = prompt.split(' ')[-1].replace('\n', '')
    rhymes.append(dic.inserted(word).split('-')[-1])

looper_count = 10000
prev_word = ''
for i in tqdm(range(60)):
    next_token, tokens_possible  = get_next_token(generated, looper_count)
    generated_tmp = torch.cat((generated, next_token), dim=1)
    next_token_tmp, _ = get_next_token(generated_tmp)
    
    curr_word = tokenizer.decode(next_token[0])
    next_word = tokenizer.decode(next_token_tmp[0])
    
    looper = 0
    
    if '\n' ==  next_word and len(rhymes) < wersy_n:
        output_list = list(generated.squeeze().numpy())
        word = tokenizer.decode(output_list).replace('\n', ' ').strip().split(' ')[-1]
        syllab = dic.inserted(re.sub('[,.-;?!]',' ',word)).split('-')[-1]            
        rhymes.append(word)    
        print('A', rhymes)
    elif '\n' ==  next_word and len(rhymes) == wersy_n:
        output_list = list(generated.squeeze().numpy())
        word = tokenizer.decode(output_list).replace('\n', ' ').strip().split(' ')[-1]
        syllab = dic.inserted(re.sub('[,.-;!?]',' ',word)).split('-')[-1]            
        org_word = word

        for looper in tqdm(range(tokens_possible.shape[0]),desc='looking'):
            word = tokenizer.decode(tokens_possible[looper])
            syllab = dic.inserted(word).split('-')[-1]
            
#             generated_tmp = torch.cat((generated, tokens_possible[looper].reshape(1,1)), dim=1)
#             next_token_tmp, _ = get_next_token(generated_tmp)

            next_word = tokenizer.decode(next_token_tmp[0])
            if rhymes[0].lower() == syllab.lower() : # and next_word == '\n':
                break

        if looper+1 < looper_count:
            print(prev_word, rhymes,word,syllab)
#             next_token = tokens_possible[looper].reshape(1,1)
            next_token = torch.cat((tokens_possible[looper].reshape(1,1),
                                    torch.tensor(tokenizer.encode('\n')).unsqueeze(0)), dim=1)
        rhymes = []
        
    prev_word = tokenizer.decode(next_token[0])

    generated = torch.cat((generated, next_token), dim=1)
    
#     output_list = list(generated.squeeze().numpy())
#     output_text = tokenizer.decode(output_list)
#     print(output_text)
    
#     if curr_word == '\n' and next_word == '\n':
#         break

  8%|███▋                                        | 5/60 [00:01<00:14,  3.79it/s]

 pomoże ['wie'] czowie wie


 32%|█████████████▌                             | 19/60 [00:04<00:10,  3.77it/s]

A ['słowa']


 47%|████████████████████                       | 28/60 [00:07<00:10,  3.08it/s]

...

 72%|██████████████████████████████▊            | 43/60 [00:13<00:06,  2.64it/s]

A ['bezpieczny,']


 87%|█████████████████████████████████████▎     | 52/60 [00:16<00:03,  2.15it/s]

A ['ratował']


100%|███████████████████████████████████████████| 60/60 [00:19<00:00,  3.01it/s]


In [95]:
output_list = list(generated.squeeze().numpy())
output_text = tokenizer.decode(output_list)
print(output_text)

Litwo Ojczyzno moja ty jesteś jak zdrowie
I łaska twoja pomożeczowie
I leliją na każdy dzień
I cisną się słowa: "Dobra to rzecz,
Moja jedyna."

I taka jest wola Twoja:
Żebych był bezpieczny, a
Ale kto cię dziś będzie ratował,
Kto dziś rano będzie spał na


In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

#Run the functions to generate the lyrics
generated_lyrics = text_generation(test_set)